In [9]:
import os
import sqlite3
from sqlite3 import Error
import pandas as pd
#os.system("osascript ../sendMessage.scpt  9142009306 'cats are cool'")

In [10]:
txtfile = open("../new.txt",'w')
txtfile.write("48 14 * * * osascript Desktop/sendMessage.scpt 9142009306 \"cats are neat\" "+"\n")
os.system("cat ../new.txt| crontab")

0

In [11]:
import datetime
now = datetime.datetime.now()
print(now.year, now.month, now.day, now.hour, now.minute, now.second)

2021 12 9 21 0 25


In [12]:
#!pip3.9 install imessage_reader

In [102]:
import datetime
import os
import random 
import sys
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import torch.nn.functional as F
import os
import sqlite3
from sqlite3 import Error
import pandas as pd
def messagetime(tx,ty,thx,thy,message):
    for i in range(2):
        minute =random.randrange(tx,ty)
        hour =random.randrange(thx,thy)
        txtfile = open("new.txt",'w')
        minute,hour = str(minute),str(hour)
        message =str(message)
        f = minute +" "+hour+" * * * osascript Desktop/sendMessage.scpt 6464203504" +" \""+message+"\" "+"\n"
        txtfile.write(f)
        os.system("cat new.txt| crontab")


def load_tokenizer_and_model(model="microsoft/DialoGPT-large"): 
    # Initialize tokenizer and model
    print("Loading model...")
    tokenizer = AutoTokenizer.from_pretrained(model)
    model = AutoModelForCausalLM.from_pretrained(model)

    # Return tokenizer and model
    return tokenizer, model


def generate_response(sen, tokenizer, model, chat_round, chat_history_ids):
    """
    Generate a response to some user input.
    """
    # Encode user input and End-of-String (EOS) token
    new_input_ids = tokenizer.encode(sen + tokenizer.eos_token, return_tensors='pt')

    # Append tokens to chat history
    try:
        bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1) 
    except:
        bot_input_ids = new_input_ids
        print("here!")

    # Generate response given maximum chat length history of 1250 tokens
    #chat_history_ids = model.generate(bot_input_ids, max_length=1250, pad_token_id=tokenizer.eos_token_id)
    chat_history_ids = model.generate(
        bot_input_ids,
        max_length=1200,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        top_k=50, 
        top_p=0.9, 
        num_return_sequences=1,
        temperature=0.5
    )

    #print the output
    output = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    # Print response
    print("AI: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

    # Return the chat history ids
    return output



def get_df():
    db_file = "/Users/sophiasackstein/Library/Messages/chat.db"
    conn = sqlite3.connect(db_file)
    cur = conn.cursor()
    p= """ 
        select
     m.rowid,
     m.handle_id
    ,coalesce(m.cache_roomnames, h.id) ThreadId
    ,m.is_from_me IsFromMe
    ,case when m.is_from_me = 1 then m.account
     else h.id end as FromPhoneNumber
    ,case when m.is_from_me = 0 then m.account
     else coalesce(h2.id, h.id) end as ToPhoneNumber
    ,m.service Service

    /*,datetime(m.date + 978307200, 'unixepoch', 'localtime') as TextDate -- date stored as ticks since 2001-01-01 */
    ,datetime((m.date / 1000000000) + 978307200, 'unixepoch', 'localtime') as TextDate /* after iOS11 date needs to be / 1000000000 */

    ,m.text MessageText

    ,c.display_name RoomName

    from
    message as m
    inner join chat_message_join T2 on m.rowid=T2.message_id 
    inner join message T1 on m.rowid=T2.message_id 
    left join handle as h on m.handle_id = h.rowid
    left join chat as c on m.cache_roomnames = c.room_name /* note: chat.room_name is not unique, this may cause one-to-many join */
    left join chat_handle_join as ch on c.rowid = ch.chat_id
    left join handle as h2 on ch.handle_id = h2.rowid

    where (h2.service is null or m.service = h2.service) AND T2.chat_id=3

    order by m.date; """
    cur.execute(p)
    r = cur.fetchall()
    df_msg = pd.DataFrame(r, columns=['id_x', 'id_y','num','isme','numsend','numrecieve','type','date','text', 'attachments'])
    df = df_msg.drop_duplicates()
    return df 
def respondpls(df):
    v =0
    chat_round =-1
    #tokenizer, model = load_tokenizer_and_model()

    # Initialize history variable
    chat_history_idsy = None
    chat_history_idsx = None
    me =[]
    you =[]
    stopdf=len(df[df.isme==0])
    s = ""
    l=""
    for i,r in df.iterrows():
        chat_round+=1
        if r["isme"] == 1:

            me.append(r['text'])
            l = '. '.join(me)
            you =[]
            print("me: ",l)
            chat_history_idsy = generate_response(s,tokenizer, model, chat_round, chat_history_idsy)


        if r["isme"] == 0:

            me=[]
            #chat_history_idsx = generate_prev(l,tokenizer, model, chat_round, chat_history_idsy)
            you.append(r['text'])
            l = '. '.join(you)
            v+=1
            print("you: ", l)
            chat_history_idsy = generate_response(l,tokenizer, model, chat_round, chat_history_idsy)
            s=l
        if v==stopdf:
            chat_history_idsy = generate_response(s,tokenizer, model, chat_round, chat_history_idsy)
            return chat_history_idsy
        
      
    return chat_history_idsy



In [103]:
#tokenizer, model = load_tokenizer_and_model()

In [105]:
message = "Getting out of work! stop worrying lol"
messagetime(tx=54,ty=59,thx=21,thy=22,message=message)

In [101]:
df=get_df()
response = respondpls(df)
response

me:  Hey da I’m checking if I can iMessage for when I go to Costa Rica
here!
AI: .
you:  Got it!😘
here!
AI: I love you!!
me:  Yay!
here!
AI: I just got mine too!
me:  Yay!. {query}
here!
AI: I got it!
you:  😊
here!
AI: I think you mean
here!
AI: What about it?


'What about it?'

In [ ]:
generate_prev(

In [15]:
df

,id_x,id_y,num,isme,numsend,numrecieve,type,date,text,attachments
0,4,3,+16464203504,1,e:615562@bah.com,+16464203504,iMessage,2021-12-09 12:54:17,Hey da I’m checking if I can iMessage for when...,None
32,5,3,+16464203504,0,+16464203504,e:615562@bah.com,iMessage,2021-12-09 12:59:12,Got it!😘,None
64,6,3,+16464203504,1,e:615562@bah.com,+16464203504,iMessage,2021-12-09 12:59:22,Yay!,None
96,27,3,+16464203504,1,e:615562@bah.com,+16464203504,iMessage,2021-12-09 18:14:02,{query},None
128,28,3,+16464203504,0,+16464203504,e:615562@bah.com,iMessage,2021-12-09 18:14:35,😊,None
160,29,3,+16464203504,1,e:615562@bah.com,+16464203504,iMessage,2021-12-09 18:14:46,Oops,None
192,30,3,+16464203504,1,e:615562@bah.com,+16464203504,iMessage,2021-12-09 18:15:03,Meant to test that on my number,None


In [6]:
message ="hello you"
messagetime(10,15,15,16,message)

In [7]:
#!17 * * * * /home/maverick/full-backup
#osascript sendMessage.scpt  9142009306 'cats are cool'

In [8]:
#!pip3.9 install gpt-2-simple

In [ ]:

import torch


import wandb

import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer


In [ ]:
#!pip3.9 install pyparsing==2.4.2

In [ ]:
from transformers import pipeline
generator = pipeline('text-generation', model='EleutherAI/gpt-neo-2.7B')
prompt = 'The thing I hate most' # can be anything
res = generator(prompt, max_length=50, do_sample=True, temperature=0.9)

In [ ]:
conversational_pipeline = pipeline("conversational")

In [ ]:
from transformers import Conversation
conversation = Conversation("Going to the movies tonight - any suggestions?")

# Steps usually performed by the model when generating a response:
# 1. Mark the user input as processed (moved to the history)
conversation.mark_processed()
# 2. Append a mode response
conversation.append_response("The Big lebowski.")

conversation.add_user_input("Is it good?")
conversation_1 = Conversation("Going to the movies tonight - any suggestions?")
conversation_2 = Conversation("What's the last book you have read?")

conversational_pipeline([conversation_1, conversation_2])

conversation_1.add_user_input("Is it an action movie?")
conversation_2.add_user_input("What is the genre of this book?")

conversational_pipeline([conversation_1, conversation_2])

In [ ]:
conversation1 = Conversation("The Doctor was going to call you directly, I need to talk. Call me when you can.")
#conversation1.append_response("Good I'll be available soon.")

conversation1.add_user_input("Just call during your hour. I'm not feeling well and I'm your dad..")
conversational_pipeline([conversation1])

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
model="microsoft/DialoGPT-large"
tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForCausalLM.from_pretrained(model)
  
# take user input
text = "Good I'll be available soon."
# encode the input and add end of string token


In [ ]:
#!pip3.9 install --upgrade torch

In [ ]:
text = "Can you call?"
# encode the input and add end of string token
input_ids = tokenizer.encode(text + tokenizer.eos_token, return_tensors="pt")
bot_input_ids = torch.cat([chat_history_ids, input_ids], dim=-1) if step > 0 else input_ids
# generate a bot response

chat_history_ids = model.generate(
    bot_input_ids,
    max_length=150,
    pad_token_id=tokenizer.eos_token_id,
)
#print the output
output = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
print(f"me: {output}")

In [ ]:
text="I'm your dad, can you show more love and care"
new_input_ids = tokenizer.encode(text + tokenizer.eos_token, return_tensors="pt") 
bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1)
# generate a bot response

chat_history_ids = model.generate(
    bot_input_ids,
    max_length=200,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id,
    top_k=50, 
    top_p=0.9, 
    num_return_sequences=1
)
#print the output
output = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
print(f"me: {output}")

In [9]:
import sqlite3
from sqlite3 import Error
db_file = "/Users/sophiasackstein/Library/Messages/chat.db"
conn = sqlite3.connect(db_file)
cur = conn.cursor()

In [122]:
def get_df():
    p= """ 
        select
     m.rowid,
     m.handle_id
    ,coalesce(m.cache_roomnames, h.id) ThreadId
    ,m.is_from_me IsFromMe
    ,case when m.is_from_me = 1 then m.account
     else h.id end as FromPhoneNumber
    ,case when m.is_from_me = 0 then m.account
     else coalesce(h2.id, h.id) end as ToPhoneNumber
    ,m.service Service

    /*,datetime(m.date + 978307200, 'unixepoch', 'localtime') as TextDate -- date stored as ticks since 2001-01-01 */
    ,datetime((m.date / 1000000000) + 978307200, 'unixepoch', 'localtime') as TextDate /* after iOS11 date needs to be / 1000000000 */

    ,m.text MessageText

    ,c.display_name RoomName

    from
    message as m
    inner join chat_message_join T2 on m.rowid=T2.message_id 
    inner join message T1 on m.rowid=T2.message_id 
    left join handle as h on m.handle_id = h.rowid
    left join chat as c on m.cache_roomnames = c.room_name /* note: chat.room_name is not unique, this may cause one-to-many join */
    left join chat_handle_join as ch on c.rowid = ch.chat_id
    left join handle as h2 on ch.handle_id = h2.rowid

    where (h2.service is null or m.service = h2.service) AND T2.chat_id=3

    order by m.date; """
    cur.execute(p)
    r = cur.fetchall()
    df_msg = pd.DataFrame(r, columns=['id_x', 'id_y','num','isme','numsend','numrecieve','type','date','text', 'attachments'])
    df = df_msg.drop_duplicates()
    return df 
def respondpls(df):
    v =0
    chat_round =-1
    tokenizer, model = load_tokenizer_and_model()

    # Initialize history variable
    chat_history_idsy = None
    chat_history_idsx = None
    me =[]
    you =[]
    stopdf=len(df[df.isme==0])
    s = ""
    l=""
    for i,r in df.iterrows():
        chat_round+=1
        if r["isme"] == 1:

            me.append(r['text'])
            l = '. '.join(me)
            you =[]
            print("me: ",l)


        if r["isme"] == 0:

            me=[]
            #chat_history_idsx = generate_prev(l,tokenizer, model, chat_round, chat_history_idsy)
            you.append(r['text'])
            l = '. '.join(you)
            v+=1
            print("you: ", l)
        s+=l
        if v==stopdf and stopdf>1:
            chat_history_idsy = generate_response(s,tokenizer, model, chat_round, chat_history_idsy)
            break
        else:
            chat_history_idsy = generate_response(s,tokenizer, model, chat_round, chat_history_idsy)
        
        return chat_history_idsy

[(4,
  3,
  '+16464203504',
  1,
  'e:615562@bah.com',
  '+16464203504',
  'iMessage',
  '2021-12-09 12:54:17',
  'Hey da I’m checking if I can iMessage for when I go to Costa Rica',
  None),
 (4,
  3,
  '+16464203504',
  1,
  'e:615562@bah.com',
  '+16464203504',
  'iMessage',
  '2021-12-09 12:54:17',
  'Hey da I’m checking if I can iMessage for when I go to Costa Rica',
  None),
 (4,
  3,
  '+16464203504',
  1,
  'e:615562@bah.com',
  '+16464203504',
  'iMessage',
  '2021-12-09 12:54:17',
  'Hey da I’m checking if I can iMessage for when I go to Costa Rica',
  None),
 (4,
  3,
  '+16464203504',
  1,
  'e:615562@bah.com',
  '+16464203504',
  'iMessage',
  '2021-12-09 12:54:17',
  'Hey da I’m checking if I can iMessage for when I go to Costa Rica',
  None),
 (4,
  3,
  '+16464203504',
  1,
  'e:615562@bah.com',
  '+16464203504',
  'iMessage',
  '2021-12-09 12:54:17',
  'Hey da I’m checking if I can iMessage for when I go to Costa Rica',
  None),
 (4,
  3,
  '+16464203504',
  1,
  'e:61

In [184]:
def get_testdf():
    p= """ 
        select
     m.rowid,
     m.handle_id
    ,coalesce(m.cache_roomnames, h.id) ThreadId
    ,m.is_from_me IsFromMe
    ,case when m.is_from_me = 1 then m.account
     else h.id end as FromPhoneNumber
    ,case when m.is_from_me = 0 then m.account
     else coalesce(h2.id, h.id) end as ToPhoneNumber
    ,m.service Service

    /*,datetime(m.date + 978307200, 'unixepoch', 'localtime') as TextDate -- date stored as ticks since 2001-01-01 */
    ,datetime((m.date / 1000000000) + 978307200, 'unixepoch', 'localtime') as TextDate /* after iOS11 date needs to be / 1000000000 */

    ,m.text MessageText

    ,c.display_name RoomName

    from
    message as m
    inner join chat_message_join T2 on m.rowid=T2.message_id 
    inner join message T1 on m.rowid=T2.message_id 
    left join handle as h on m.handle_id = h.rowid
    left join chat as c on m.cache_roomnames = c.room_name /* note: chat.room_name is not unique, this may cause one-to-many join */
    left join chat_handle_join as ch on c.rowid = ch.chat_id
    left join handle as h2 on ch.handle_id = h2.rowid

    where (h2.service is null or m.service = h2.service) AND T2.chat_id=2

    order by m.date; """
    cur.execute(p)
    r = cur.fetchall()
    df_msg = pd.DataFrame(r, columns=['id_x', 'id_y','num','isme','numsend','numrecieve','type','date','text', 'attachments'])
    df = df_msg.drop_duplicates()
    return df 

In [189]:
def respondpls(df):
    v =0
    chat_round =-1
    tokenizer, model = load_tokenizer_and_model()

    # Initialize history variable
    chat_history_idsy = None
    chat_history_idsx = None
    me =[]
    you =[]
    stopdf=len(df[df.isme==0])
    s = ""
    l=""
    for i,r in df.iterrows():
        chat_round+=1
        if r["isme"] == 1:

            me.append(r['text'])
            l = '. '.join(me)
            you =[]
            print("me: ",l)


        if r["isme"] == 0:

            me=[]
            #chat_history_idsx = generate_prev(l,tokenizer, model, chat_round, chat_history_idsy)
            you.append(r['text'])
            l = '. '.join(you)
            v+=1
            print("you: ", l)
        s+=l
        if v==stopdf and stopdf>1:
            chat_history_idsy = generate_response(s,tokenizer, model, chat_round, chat_history_idsy)
            break
        else:
            chat_history_idsy = generate_response(s,tokenizer, model, chat_round, chat_history_idsy)
        
        return chat_history_idsy

In [190]:
df= get_testdf()
respondpls(df)

Loading model...
me:  Hello
AI: hi. u can pm me anytime.


'hi. u can pm me anytime.'

In [123]:
df_msg = pd.DataFrame(r, columns=['id_x', 'id_y','num','isme','numsend','numrecieve','type','date','text', 'attachments'])

In [124]:
df = df_msg.drop_duplicates()
df

,id_x,id_y,num,isme,numsend,numrecieve,type,date,text,attachments
0,4,3,+16464203504,1,e:615562@bah.com,+16464203504,iMessage,2021-12-09 12:54:17,Hey da I’m checking if I can iMessage for when...,None
30,5,3,+16464203504,0,+16464203504,e:615562@bah.com,iMessage,2021-12-09 12:59:12,Got it!😘,None
60,6,3,+16464203504,1,e:615562@bah.com,+16464203504,iMessage,2021-12-09 12:59:22,Yay!,None
90,27,3,+16464203504,1,e:615562@bah.com,+16464203504,iMessage,2021-12-09 18:14:02,{query},None
120,28,3,+16464203504,0,+16464203504,e:615562@bah.com,iMessage,2021-12-09 18:14:35,😊,None
150,29,3,+16464203504,1,e:615562@bah.com,+16464203504,iMessage,2021-12-09 18:14:46,Oops,None
180,30,3,+16464203504,1,e:615562@bah.com,+16464203504,iMessage,2021-12-09 18:15:03,Meant to test that on my number,None


In [125]:
df = df.reset_index()
df

,index,id_x,id_y,num,isme,numsend,numrecieve,type,date,text,attachments
0,0,4,3,+16464203504,1,e:615562@bah.com,+16464203504,iMessage,2021-12-09 12:54:17,Hey da I’m checking if I can iMessage for when...,None
1,30,5,3,+16464203504,0,+16464203504,e:615562@bah.com,iMessage,2021-12-09 12:59:12,Got it!😘,None
2,60,6,3,+16464203504,1,e:615562@bah.com,+16464203504,iMessage,2021-12-09 12:59:22,Yay!,None
3,90,27,3,+16464203504,1,e:615562@bah.com,+16464203504,iMessage,2021-12-09 18:14:02,{query},None
4,120,28,3,+16464203504,0,+16464203504,e:615562@bah.com,iMessage,2021-12-09 18:14:35,😊,None
5,150,29,3,+16464203504,1,e:615562@bah.com,+16464203504,iMessage,2021-12-09 18:14:46,Oops,None
6,180,30,3,+16464203504,1,e:615562@bah.com,+16464203504,iMessage,2021-12-09 18:15:03,Meant to test that on my number,None


In [135]:
df[df.isme==0]

,index,id_x,id_y,num,isme,numsend,numrecieve,type,date,text,attachments
1,30,5,3,+16464203504,0,+16464203504,e:615562@bah.com,iMessage,2021-12-09 12:59:12,Got it!😘,None
4,120,28,3,+16464203504,0,+16464203504,e:615562@bah.com,iMessage,2021-12-09 18:14:35,😊,None


In [171]:
v =0
chat_round =-1
tokenizer, model = load_tokenizer_and_model()

# Initialize history variable
chat_history_idsy = None
chat_history_idsx = None
me =[]
you =[]
stopdf=len(df[df.isme==0])
s = ""
l=""
for i,r in df.iterrows():
    chat_round+=1
    if r["isme"] == 1:
        
        me.append(r['text'])
        l = '. '.join(me)
        you =[]
        print("me: ",l)
    
        
    if r["isme"] == 0:
        
        me=[]
        #chat_history_idsx = generate_prev(l,tokenizer, model, chat_round, chat_history_idsy)
        you.append(r['text'])
        l = '. '.join(you)
        v+=1
        print("you: ", l)
    s+=l
    s+="Love you, my wonderful smart daughter!"
    s+="Remember to call your dad once in a while!"
    if v==stopdf:
        chat_history_idsy = generate_response(s,tokenizer, model, chat_round, chat_history_idsy)
        break

Loading model...
me:  Hey da I’m checking if I can iMessage for when I go to Costa Rica
you:  Got it!😘
me:  Yay!
me:  Yay!. {query}
you:  😊
AI: Love you<|endoftext|>


In [174]:
generate_response("hello",tokenizer, model, chat_round+1, chat_history_ids)

AI: me too


tensor([[ 4211,  2185,    25, 14690, 12379,   314,   447,   247,    76, 10627,
           611,   314,   460,  1312, 12837,   329,   618,   314,   467,   284,
         18133, 31656, 50256,    40,  1101,   287, 18133, 31656,   290,   314,
           460,   470,  1064,   428,  3895,   764, 50256,  4211,   921,    25,
         11853,   340,     0, 47249,   246, 50256,    40,  1101,   287, 18133,
         31656,   290,   314,   460,   470,  1064,   428,  3895,   764, 50256,
          4211,  2185,    25,   575,   323,     0, 50256,  5308,  1165,  5145,
         50256,  4211,  2185,    25,   575,   323, 43179,    90, 22766,    92,
         50256,  5308,  1058,   575,   323,  5145, 50256,  4211,   921,    25,
         30325,   232, 50256,  5308,  1058,   575,   323,  5145, 50256,  4211,
          2185,    25, 48002, 50256,  5308,  1058,   575,   323,  5145, 50256,
          4211,  2185,    25, 48002,    13,  5308,   415,   284,  1332,   326,
           319,   616,  1271, 50256,  5308,  1058,  